In [16]:
# get the data
import pandas as pd
from sklearn.model_selection import train_test_split as ttt

# first we get the data
testPath = "Data/test.csv"
trainPath = "Data/train.csv"
genderSubmissionPath = "Data/gender_submission.csv"

testPDRaw = pd.read_csv(testPath)
trainPDRaw = pd.read_csv(trainPath)
genderSubPDRaw = pd.read_csv(genderSubmissionPath)

# merge the test data too
testDataMerge= pd.merge(genderSubPDRaw,testPDRaw,on = 'PassengerId')


In [17]:
# merge test and train data in order to do the same changes
combine = [trainPDRaw,testPDRaw]
# unuseful data might be the passanger ticket
# remove passanger ticket
for dataset in combine:
    dataset.drop( columns= ['Ticket'], inplace = True)


In [ ]:
# considering the names we could reduce it Mr, Mrs, Miss, Col, Master to make it easier
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)



In [31]:
# pd.crosstab(trainPDRaw.Title,trainPDRaw.Survived)
# we replace: mlle and Ms by miss, 
for dataset in combine:
    dataset.Title.replace('Ms', 'Miss', inplace = True)
    dataset.Title.replace('Mme', 'Miss', inplace = True)
    dataset.Title.replace(['Capt','Col','Countess','Don','Dr','Jonkheer','Major','Lady','Rev','Sir'], 'Rare', inplace = True)

pd.crosstab(trainPDRaw.Title, trainPDRaw.Sex)
# print(testPDRaw.head)

Sex,female,male
Title,,
Master,0,40
Miss,186,0
Mr,0,517
Mrs,125,0
Rare,3,20


In [ ]:
# see where are we at
# now that we have modified the name into title we drop the name column

for data in combine:
    data.drop(columns= ['Name'], inplace = True)

In [ ]:
# we replace male and female by 1 and 0
for data in combine:
    data.Sex.replace('female', '1',inplace=True)
    data.Sex.replace('male','0', inplace =True)
    

In [42]:
# we raw the histogram for the fares
import matplotlib.pyplot as plt

# plt.hist(combine[0].Fare)
combine[0].Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64